In [ ]:
import numpy as np
import torch
import torch.nn as nn
from gensim.models import KeyedVectors
import re, string
!pip install pyvi
from pyvi import ViTokenizer

# 1) LOAD WORD EMBEDDING & TIỀN XỬ LÝ


file_embedding = "cc.vi.300.vec"
print("Đang load embedding...")
model_emb = KeyedVectors.load_word2vec_format(file_embedding, binary=False)
embedding_dim = model_emb.vector_size

def get_vector(token):
    # Trả về vector embedding của token
    if token in model_emb.key_to_index:
        return model_emb[token]
    else:
        return np.zeros(embedding_dim, dtype=np.float32)

def preprocess_vi(sentence):

    s = sentence.lower().strip()
    # tokenize
    s = ViTokenizer.tokenize(s)
    tokens = s.split()
    return tokens

# 2) HÀM SEMANTIC MATCHING & DECOMPOSITION

def cosine_sim(a, b):
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    if norm_a < 1e-9 or norm_b < 1e-9:
        return 0.0
    return float((a @ b)/(norm_a*norm_b))

def find_best_match(vec_s, list_vec_t):
    # matching: chọn t_j có cos-sim cao nhất
    best_sim = -1.0
    best_vec = np.zeros_like(vec_s)
    for vec_t in list_vec_t:
        sim = cosine_sim(vec_s, vec_t)
        if sim > best_sim:
            best_sim = sim
            best_vec = vec_t
    return best_vec

def linear_decompose(s_i, s_i_hat):
    # alpha = cos(s_i, s_i_hat)
    alpha = cosine_sim(s_i, s_i_hat)
    s_plus = alpha * s_i
    s_minus = (1.0 - alpha) * s_i
    return s_plus, s_minus

# 3) KIẾN TRÚC MẠNG 2-CHANNEL CNN (giống lúc train)

class TwoChannelCNN(nn.Module):
    def __init__(self, emb_dim=300, num_filters=64, kernel_size=3):
        super().__init__()
        self.conv = nn.Conv2d(
            in_channels=2,
            out_channels=num_filters,
            kernel_size=(kernel_size, kernel_size),
            padding=(1,1)
        )
        self.pool = nn.AdaptiveMaxPool2d((1,1))
        self.fc = nn.Linear(num_filters, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # x shape: (B, 2, seq_len, emb_dim)
        feat = self.conv(x) 
        feat = nn.functional.relu(feat)
        pooled = self.pool(feat)
        # => (B, num_filters, 1, 1)
        flattened = pooled.squeeze(-1).squeeze(-1) # => (B, num_filters)
        out = self.fc(flattened)                  # => (B,1)
        out = self.sigmoid(out)                   # => (B,1) ∈ [0,1]
        return out

# 4) HÀM predict_similarity(model, sent1, sent2)
#    Thực hiện pipeline: 
#      - Preprocess -> get embedding
#      - matching + decomposition
#      - ghép 2 channel -> CNN -> similarity

def predict_similarity(model, sentence1, sentence2, max_len=20):
    # 1) Preprocess
    tokens_s = preprocess_vi(sentence1)
    tokens_t = preprocess_vi(sentence2)
    
    # 2) Embed
    vecs_s = [get_vector(w) for w in tokens_s]
    vecs_t = [get_vector(w) for w in tokens_t]
    
    # 3) Matching + Decomposition
    s_plus_list = []
    s_minus_list = []
    for s_i in vecs_s:
        s_i_hat = find_best_match(s_i, vecs_t)
        s_plus, s_minus = linear_decompose(s_i, s_i_hat)
        s_plus_list.append(s_plus)
        s_minus_list.append(s_minus)
    
    # 4) Truncate / pad cho đủ max_len
    s_plus_list  = s_plus_list[:max_len]
    s_minus_list = s_minus_list[:max_len]
    
    pad_len = max_len - len(s_plus_list)
    s_plus_list  += [np.zeros(embedding_dim, dtype=np.float32)] * pad_len
    s_minus_list += [np.zeros(embedding_dim, dtype=np.float32)] * pad_len
    
    # (seq_len, emb_dim) => stack => (max_len, emb_dim)
    plus_array  = np.stack(s_plus_list, axis=0)
    minus_array = np.stack(s_minus_list, axis=0)
    
    # 5) Ghép 2 kênh => shape (1,2,max_len,emb_dim)
    sim_tensor = torch.tensor([plus_array], dtype=torch.float)
    dis_tensor = torch.tensor([minus_array], dtype=torch.float)
    
    input_tensor = torch.stack([sim_tensor, dis_tensor], dim=1)
    
    # 6) Đưa vào model -> ra similarity
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
    # output shape (1,1)
    sim_score = float(output.item())
    return sim_score




Defaulting to user installation because normal site-packages is not writeable
Đang load embedding...


In [ ]:
import csv

# Khởi tạo dictionary
qa_dict = {}

with open("q&a-2.csv", "r", encoding="utf-8") as file:
    reader = csv.reader(file, delimiter=';')
    for row in reader:
        question, answer = row
        if question in qa_dict:
            # Nếu câu hỏi đã tồn tại, thêm câu trả lời vào danh sách
            qa_dict[question].append(answer)
        else:
            # Nếu câu hỏi chưa tồn tại, tạo danh sách mới
            qa_dict[question] = [answer]


In [183]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    
    model = TwoChannelCNN(emb_dim=embedding_dim, num_filters=64, kernel_size=3).to(device)
    
    model.load_state_dict(torch.load("model_weights_vi_6.pt", map_location=device))

    sentA = "hang động băng được hình thành như thế nào"
    sentB = "Động cơ hướng tâm của máy bay được chế tạo như thế nào"
    
    similarity_value = predict_similarity(model, sentA, sentB)
    print(f"Similarity giữa 2 câu: {similarity_value:.4f}")

Using device: cpu
Similarity giữa 2 câu: 0.5788


/var/folders/3z/mk4zz8y5633d57znqmn946740000gn/T/ipykernel_2897/3315487751.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model_weight

In [188]:
threshold = 0.7

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    # Load the model
    model = TwoChannelCNN(emb_dim=embedding_dim, num_filters=64, kernel_size=3).to(device)
    model.load_state_dict(torch.load("model_weights_vi_6.pt", map_location=device))

    # Sample input
    sentA = input("Enter your question: ")

    # Compute similarity for each key
    similarities = []
    for key in qa_dict.keys():
        similarity_value = predict_similarity(model, sentA, key)
        if similarity_value > threshold:
            similarities.append((key, similarity_value))

    # Check if similarities list is empty
    if len(similarities) == 0:  # Use len(similarities) to check the length of the list
        print("No questions found in the dataset matching your question!")
    else:
        # Sort by similarity value in descending order
        top_keys = sorted(similarities, key=lambda x: x[1], reverse=True)[:5]

        # Display the top 5 keys with highest similarity along with their answers
        print("\nQuestion: " + sentA)
        print("\nTop most similar questions with answers:")
        for i, (key, sim_value) in enumerate(top_keys, start=1):
            print(f"{i}. {key} - Similarity: {sim_value:.2f}")
            print("   Answers:")
            for answer in qa_dict[key]:
                print(f"     - {answer}")


/var/folders/3z/mk4zz8y5633d57znqmn946740000gn/T/ipykernel_2897/3779328771.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model_weight

Using device: cpu



Question: hang động được hình thành như thế nào

Top most similar questions with answers:
1. hang động sông băng được hình thành như thế nào? - Similarity: 0.75
   Answers:
     - Một hang động sông băng chìm một phần trên sông băng Perito Moreno.
     - Mặt tiền băng cao khoảng 60 m
     - Các khối băng trong hang động sông băng Titlis
     - Hang động sông băng là hang động được hình thành bên trong lớp băng của sông băng.
     - Hang động sông băng thường được gọi là hang động băng, nhưng thuật ngữ này được sử dụng chính xác để mô tả các hang động nền đá chứa băng quanh năm.
